In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
    !pip install --upgrade xee

In [ ]:
!pip install rioxarray

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path1 = '/content/drive/MyDrive/Amini'

In [ ]:
import os
import glob

# search for .tif files
tif_files = glob.glob(os.path.join(path1, '*.tif'))

# Print the list of .tif files
if tif_files:
    print("Found .tif files:")
    for file in tif_files:
        print(file)
else:
    print("No .tif files found in the directory.")


In [ ]:
import os
import re
import rioxarray as rxr
import xarray as xr
import numpy as np

# Define your data directory
data_dir = "/content/drive/MyDrive/Amini"

# Find all .tif files in the directory
tif_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".tif")]

# Regular expressions to match time periods and variable names
time_pattern = re.compile(r"(Dec_2024|Jan_2025|Feb_2025)")
var_pattern = re.compile(r"(Sentinel1_VV|Sentinel1_VH|NDVI|LST)")

# Dictionary to store datasets
datasets = {}

print("\n Found .tif files:")
for file in tif_files:
    filename = os.path.basename(file)
    time_match = time_pattern.search(filename)
    var_match = var_pattern.search(filename)

    if time_match and var_match:
        time_label = time_match.group(1)
        var_label = var_match.group(1)

        # Create a nested dictionary structure
        if time_label not in datasets:
            datasets[time_label] = {}

        # Load the raster dataset
        try:
            ds = rxr.open_rasterio(file).squeeze()
            ds = ds.rename("value")  # Ensure naming consistency
            datasets[time_label][var_label] = ds
            print(f" Matched: {filename} → Time: {time_label}, Variable: {var_label}")
        except Exception as e:
            print(f" Error loading {filename}: {e}")

    else:
        print(f"⚠ Skipped: {filename} (No match on time/variable regex)")

# Check if we have successfully organized datasets
print("\n Organized Datasets:")
for time_label, variables in datasets.items():
    print(f" {time_label}: {list(variables.keys())}")

#  Check dataset dimensions before stacking
print("\n Dataset Dimensions Before Stacking:")
for time_label, variables in datasets.items():
    for var_label, ds in variables.items():
        print(f" {time_label} - {var_label}: {ds.shape}")

# Ensure all variables exist for each time period (avoid missing data issues)
all_variables = {"Sentinel1_VV", "Sentinel1_VH", "NDVI", "LST"}
for time_label in datasets.keys():
    missing_vars = all_variables - set(datasets[time_label].keys())
    for var in missing_vars:
        print(f"⚠ Warning: {time_label} is missing {var}. Filling with NaNs.")
        example_shape = next(iter(datasets[time_label].values())).shape
        datasets[time_label][var] = xr.DataArray(np.full(example_shape, np.nan), dims=("y", "x"))

#  Stack datasets into a single xarray DataArray
stacked_datasets = []
time_labels = sorted(datasets.keys())  # Sort to maintain proper time order

for time_label in time_labels:
    try:
        time_ds = xr.concat([datasets[time_label][var] for var in sorted(all_variables)], dim="variable")
        time_ds = time_ds.assign_coords(time=time_label, variable=list(sorted(all_variables)))
        stacked_datasets.append(time_ds)
    except ValueError as e:
        print(f" Error concatenating datasets for {time_label}: {e}")

# Final merging of all time periods
try:
    final_datacube = xr.concat(stacked_datasets, dim="time")
    final_datacube = final_datacube.assign_coords(time=time_labels)  # Ensure proper time labels
    print("\n Successfully stacked all datasets into a DataArray!")
except ValueError as e:
    print(f" Final concatenation error: {e}")
    final_datacube = None

# Save the final dataset as NetCDF
if final_datacube is not None and not final_datacube.isnull().all():
    output_path = os.path.join(data_dir, "datacube.nc")
    final_datacube.to_netcdf(output_path)
    print(f"\n Saved final dataset to: {output_path}")
else:
    print("\n No final dataset created due to errors.")


In [ ]:
# import os
# import re
# import rioxarray as rxr
# import xarray as xr
# import numpy as np

# # Define your data directory
# data_dir = "/content/drive/MyDrive/Amini Africa"

# # Find all .tif files in the directory
# tif_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".tif")]

# # Regular expressions to match time periods and variable names
# time_pattern = re.compile(r"(Dec_2024|Jan_2025|Feb_2025)")
# var_pattern = re.compile(r"(Sentinel1_VV|Sentinel1_VH|NDVI|LST|SRTM_DEM)")

# # Dictionary to store datasets
# datasets = {}

# print("\nFound .tif files:")
# for file in tif_files:
#     filename = os.path.basename(file)
#     time_match = time_pattern.search(filename)
#     var_match = var_pattern.search(filename)

#     if time_match and var_match:
#         time_label = time_match.group(1)
#         var_label = var_match.group(1)

#         # Create a nested dictionary structure
#         if time_label not in datasets:
#             datasets[time_label] = {}

#         # Load the raster dataset
#         try:
#             ds = rxr.open_rasterio(file).squeeze()
#             ds = ds.rename("value")  # Ensure naming consistency
#             datasets[time_label][var_label] = ds
#             print(f"Matched: {filename} → Time: {time_label}, Variable: {var_label}")
#         except Exception as e:
#             print(f"Error loading {filename}: {e}")

#     else:
#         print(f"⚠ Skipped: {filename} (No match on time/variable regex)")

# # Check if we have successfully organized datasets
# print("\nOrganized Datasets:")
# for time_label, variables in datasets.items():
#     print(f"{time_label}: {list(variables.keys())}")

# # Check dataset dimensions before stacking
# print("\nDataset Dimensions Before Stacking:")
# for time_label, variables in datasets.items():
#     for var_label, ds in variables.items():
#         print(f"{time_label} - {var_label}: {ds.shape}")

# # Ensure all variables exist for each time period (avoid missing data issues)
# all_variables = {"Sentinel1_VV", "Sentinel1_VH", "NDVI", "LST", "SRTM_DEM"}
# for time_label in datasets.keys():
#     missing_vars = all_variables - set(datasets[time_label].keys())
#     for var in missing_vars:
#         print(f"⚠ Warning: {time_label} is missing {var}. Filling with NaNs.")
#         example_shape = next(iter(datasets[time_label].values())).shape
#         datasets[time_label][var] = xr.DataArray(np.full(example_shape, np.nan), dims=("y", "x"))

# # Load the SRTM DEM file separately (since it's static across time)
# srtm_files = [file for file in tif_files if "SRTM_DEM" in file]

# if srtm_files:
#     dem_path = srtm_files[0]  # Take the first found DEM file
#     print(f"\nFound SRTM DEM: {dem_path}")

#     try:
#         dem_ds = rxr.open_rasterio(dem_path).squeeze()
#         dem_ds = dem_ds.rename("SRTM_DEM")
#     except Exception as e:
#         print(f"Error loading SRTM DEM: {e}")
#         dem_ds = None
# else:
#     print("\nNo SRTM DEM file found!")
#     dem_ds = None

# # Ensure SRTM DEM is included for each time period
# if dem_ds is not None:
#     for time_label in datasets.keys():
#         datasets[time_label]["SRTM_DEM"] = dem_ds

# # Stack datasets into a single xarray DataArray
# stacked_datasets = []
# time_labels = sorted(datasets.keys())  # Sort to maintain proper time order

# for time_label in time_labels:
#     try:
#         time_ds = xr.concat([datasets[time_label][var] for var in sorted(all_variables)], dim="variable")
#         time_ds = time_ds.assign_coords(time=time_label, variable=list(sorted(all_variables)))
#         stacked_datasets.append(time_ds)
#     except ValueError as e:
#         print(f"Error concatenating datasets for {time_label}: {e}")

# # Final merging of all time periods
# try:
#     final_datacube = xr.concat(stacked_datasets, dim="time")
#     final_datacube = final_datacube.assign_coords(time=time_labels)  # Ensure proper time labels
#     print("\nSuccessfully stacked all datasets into a DataArray!")
# except ValueError as e:
#     print(f"Final concatenation error: {e}")
#     final_datacube = None

# # Save the final dataset as NetCDF
# if final_datacube is not None and not final_datacube.isnull().all():
#     output_path = os.path.join(data_dir, "datacube.nc")
#     final_datacube.to_netcdf(output_path)
#     print(f"\nSaved final dataset to: {output_path}")
# else:
#     print("\nNo final dataset created due to errors.")


In [ ]:
# Access data for a specific time step/ to check structure (e.g., "Jan_2025")
time_step = "Jan_2025"
time_data = final_datacube.sel(time=time_step)

print(f"\nData for {time_step}:\n")
print(time_data)


In [ ]:
# Check the available time labels/ steps
print(final_datacube.time.values)


In [ ]:
# Select the data for December 2024
december_data = final_datacube.sel(time='Dec_2024')

# Print available variables for December
print("Available variables for Dec_2024:", december_data.variable.values)


In [ ]:
import matplotlib.pyplot as plt

# Set up the figure with 1 row and 2 columns for the plots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# List of variables and corresponding color maps
variables = ["LST", "NDVI"]
cmaps = ["inferno", "RdYlGn"]  # Different color schemes for LST and NDVI

# Loop through the variables and plot them
for i, (var, cmap) in enumerate(zip(variables, cmaps)):
    ax = axes[i]  # Get the current subplot axis
    data = final_datacube.sel(time="Dec_2024", variable=var)  # Select data for Dec 2024 and the current variable

    # Plot the data with the chosen colormap
    im = data.plot(ax=ax, cmap=cmap)

    # Update the colorbar to avoid incorrect labels
    cbar = im.colorbar
    cbar.set_label(var)  # Set the correct label for the colorbar

    ax.set_title(f"{var} - Dec 2024")  # Set the title for each subplot

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:
print(data)


In [ ]:
print(data.coords["variable"].values) #check for visualization of NDVI variables


In [ ]:
print(data.coords["time"].values)


In [ ]:
import xarray as xr

# Load the NetCDF datacube
file_path = "/content/drive/MyDrive/Amini/datacube.nc"
data = xr.open_dataset(file_path)

# Print the dataset structure
print(data)

# Check the number of time steps
if "time" in data.dims:
    print(f"Number of time steps: {data.sizes['time']}")
else:
    print("No 'time' dimension found in the datacube.")


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
dataset = xr.open_dataset("/content/drive/MyDrive/Amini/datacube.nc")

# Convert time labels to proper datetime format
time_str = dataset["time"].values  # ['Dec_2024', 'Feb_2025', 'Jan_2025']
time_dt = pd.to_datetime(time_str, format="%b_%Y")  # Convert to datetime

# Assign back to dataset for correct ordering
dataset = dataset.assign_coords(time=("time", time_dt))

# Sort dataset by time
dataset = dataset.sortby("time")

# Select NDVI as a DataArray
ndvi_data = dataset.sel(variable="NDVI")["value"]  # Extract NDVI values

# Extract sorted time values
ndvi_times = ndvi_data["time"].values

# Create subplots
fig, axes = plt.subplots(1, len(ndvi_times), figsize=(15, 5))

# If there's only one time step, make axes iterable
if len(ndvi_times) == 1:
    axes = [axes]

for i, time in enumerate(ndvi_times):
    ax = axes[i]
    ax.set_title(f"NDVI on {pd.to_datetime(str(time)).strftime('%b %Y')}")

    # Plot NDVI for each time step
    img = ax.imshow(ndvi_data.sel(time=time), cmap="RdYlGn", vmin=-1, vmax=1)
    plt.colorbar(img, ax=ax, label="NDVI")

plt.tight_layout()
plt.show()


In [ ]:
import xarray as xr
import pandas as pd

# Load the dataset
dataset = xr.open_dataset("/content/drive/MyDrive/Amini/datacube.nc")

# Convert time labels to proper datetime format
time_str = dataset["time"].values  # Example: ['Dec_2024', 'Feb_2025', 'Jan_2025']
time_dt = pd.to_datetime(time_str, format="%b_%Y")  # Convert to datetime

# Assign back to dataset for correct ordering
dataset = dataset.assign_coords(time=("time", time_dt))

# Sort dataset by time
dataset = dataset.sortby("time")

# Print dataset structure
print(dataset)

# Loop through all variables and print corresponding time steps
for var in dataset["variable"].values:
    data_array = dataset.sel(variable=var)["value"]
    print(f"\nVariable: {var}")
    for time in dataset["time"].values:
        shape = data_array.sel(time=time).shape
        print(f"  Time: {pd.to_datetime(str(time)).strftime('%b %Y')}, Shape: {shape}")


In [ ]:
# Load the dataset
ds = xr.open_dataset("/content/drive/MyDrive/Amini/datacube.nc")


In [ ]:
import xarray as xr

# Display dataset summary
print(ds)

# List all variables in the dataset
print(ds.variables)

# Show the first few time values to check format
print(ds.time.values)

# Show sample data for NDVI and LST_Celsius (if they exist)
if "NDVI" in ds.variables:
    print(ds["NDVI"])
if "LST_Celsius" in ds.variables:
    print(ds["LST_Celsius"])


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Ensure dataset is computed
ds = ds.compute()

# Convert time coordinate to datetime format
ds = ds.assign_coords(time=pd.to_datetime(ds.time, format="%b_%Y"))

# Select NDVI data
ds_ndvi = ds.sel(variable="NDVI").astype(np.float32)

# Create figure
fig, ax = plt.subplots(figsize=(8, 6))

# Initialize NDVI map with imshow instead of xarray plot
ndvi_plot = ax.imshow(ds_ndvi.isel(time=0).value, cmap='RdYlGn', vmin=-1, vmax=1)

# Add colorbar manually
cbar = fig.colorbar(ndvi_plot, ax=ax)
cbar.set_label("NDVI")

ax.set_title(f"NDVI Map - {ds_ndvi.time.values[0]}")

# Update function for animation
def update(num):
    ndvi_plot.set_array(ds_ndvi.isel(time=num).value.values)  # Update NDVI values
    ax.set_title(f"NDVI Map - {pd.to_datetime(str(ds_ndvi.time.values[num])).strftime('%b %Y')}")
    return ndvi_plot,

# Create animation
ani = animation.FuncAnimation(fig, update, frames=len(ds_ndvi.time), interval=500, blit=False, repeat=True)

# Save animation as MP4
ani.save("ndvi_spatial_animation.mp4", writer="ffmpeg", fps=1)

# Display animation inline (for Jupyter Notebooks)
HTML(ani.to_html5_video())
